# Building Semantic Memory with Embeddings

In this notebook, we show how to use [Chroma](https://www.trychroma.com/) with Semantic Kernel to create even more
intelligent applications. We assume that you are already familiar with the concepts of Semantic Kernel
and memory. [Previously](04-kernel-arguments-chat.ipynb), we have used `kernel arguments` to pass
additional text into prompts, enriching them with more context for a basic chat experience.

However, relying solely on kernel arguments has its limitations, such as the model's token limit.
To overcome these limitations, we will use **SK Semantic Memory**, leveraging Chroma as a persistent
Semantic Memory Storage.

**Chroma** is an open-source embedding database designed to make it easy to build Language Model
applications by making knowledge, facts, and plugins pluggable for LLMs. It allows us to store and
retrieve information in a way that can be easily utilized by the models, enabling both short-term
and long-term memory for more advanced applications. In this notebook, we will showcase how to
effectively use Chroma with the Semantic Kernel for a powerful application experience.

**Note:** This example is verified using Chroma version **0.4.10**. Any higher versions may introduce incompatibility.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.19.0"
#r "nuget: Microsoft.SemanticKernel.Connectors.Chroma, 1.19.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.19.0-alpha"
#r "nuget: System.Linq.Async, 6.0.1"
#r "nuget: Azure.Identity, 1.4.0"

#!import config/Settings.cs

using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.Chroma;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;
using Kernel = Microsoft.SemanticKernel.Kernel;
using Azure.Identity;

var builder = Kernel.CreateBuilder();
DefaultAzureCredentialOptions defaultAzureCredentialOptions = new DefaultAzureCredentialOptions { ExcludeSharedTokenCacheCredential = true, ExcludeEnvironmentCredential = true, ExcludeAzurePowerShellCredential = true, ExcludeInteractiveBrowserCredential = true, ExcludeVisualStudioCredential = true, ExcludeManagedIdentityCredential = true, ExcludeVisualStudioCodeCredential = true, ExcludeAzureCliCredential = false };

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, new DefaultAzureCredential(defaultAzureCredentialOptions));
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Azure.Identity, 1.4.0 Microsoft.SemanticKernel, 1.19.0 Microsoft.SemanticKernel.Connectors.Chroma, 1.19.0-alpha Microsoft.SemanticKernel.Plugins.Memory, 1.19.0-alpha System.Linq.Async, 6.0.1

In order to use memory, we need to instantiate the Memory Plugin with a Memory Storage
and an Embedding backend. In this example, we make use of the `ChromaMemoryStore`,
leveraging [Chroma](https://www.trychroma.com/), an open source embedding database
you can run locally and in the cloud.

To run Chroma locally, here's a quick script to download Chroma source and run it using Docker:

```shell
git clone https://github.com/chroma-core/chroma.git
cd chroma
docker-compose up --build
```

In [2]:
#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0020, SKEXP0050

using Microsoft.SemanticKernel.Connectors.AzureOpenAI;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Embeddings;
using Microsoft.SemanticKernel.Connectors.Chroma;
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

var modelId = "text-embedding-ada-002";
ITextEmbeddingGenerationService textEmbeddingService = useAzureOpenAI
    ? new AzureOpenAITextEmbeddingGenerationService(deploymentName: modelId, endpoint: azureEndpoint, credential: new DefaultAzureCredential(defaultAzureCredentialOptions))
    : new OpenAITextEmbeddingGenerationService(modelId: modelId, apiKey: apiKey);

var memory = new SemanticTextMemory(new ChromaMemoryStore("http://localhost:8000"), textEmbeddingService);

At its core, Semantic Memory is a set of data structures that allows to store
the meaning of text that come from different data sources, and optionally to
store the source text and other metadata.

The text can be from the web, e-mail providers, chats, a database, or from your
local directory, and are hooked up to the Semantic Kernel through memory connectors.

The texts are embedded, sort of "compressed", into a vector of floats that representing
mathematically the text content and meaning.

You can read more about embeddings [here](https://aka.ms/sk/embeddings).

### Manually adding memories

Let's create some initial memories "About Me". We can add memories to `ChromaMemoryStore` by using `SaveInformationAsync`

In [3]:
const string MemoryCollectionName = "aboutMe";

await memory.SaveInformationAsync(MemoryCollectionName, id: "info1", text: "My name is Andrea");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info2", text: "I currently work as a tourist operator");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info3", text: "I currently live in Seattle and have been living there since 2005");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info4", text: "I visited France and Italy five times since 2015");
await memory.SaveInformationAsync(MemoryCollectionName, id: "info5", text: "My family is from New York");

Let's try searching the memory:

In [4]:
var questions = new[]
{
    "what is my name?",
    "where do I live?",
    "where is my family from?",
    "where have I travelled?",
    "what do I do for work?",
};

foreach (var q in questions)
{
    var response = await memory.SearchAsync(MemoryCollectionName, q, limit: 1, minRelevanceScore: 0.5).FirstOrDefaultAsync();
    Console.WriteLine(q + " " + response?.Metadata.Text);
}

what is my name? My name is Andrea
where do I live? I currently live in Seattle and have been living there since 2005
where is my family from? My family is from New York
where have I travelled? I visited France and Italy five times since 2015
what do I do for work? I currently work as a tourist operator


Let's now revisit our chat sample from the [previous notebook](04-kernel-arguments-chat.ipynb).
If you remember, we used kernel arguments to fill the prompt with a `history` that continuously got populated as we chatted with the bot. Let's add also memory to it!

This is done by using the `TextMemoryPlugin` which exposes the `recall` native function.

`recall` takes an input ask and performs a similarity search on the contents that have
been embedded in the Memory Store. By default, `recall` returns the most relevant memory.

In [5]:
#pragma warning disable SKEXP0001, SKEXP0050

// TextMemoryPlugin provides the "recall" function
kernel.ImportPluginFromObject(new TextMemoryPlugin(memory));

In [6]:
const string skPrompt = @"
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

Information about me, from previous conversations:
- {{$fact1}} {{recall $fact1}}
- {{$fact2}} {{recall $fact2}}
- {{$fact3}} {{recall $fact3}}
- {{$fact4}} {{recall $fact4}}
- {{$fact5}} {{recall $fact5}}

Chat:
{{$history}}
User: {{$userInput}}
ChatBot: ";

var chatFunction = kernel.CreateFunctionFromPrompt(skPrompt, new OpenAIPromptExecutionSettings { MaxTokens = 200, Temperature = 0.8 });

The `RelevanceParam` is used in memory search and is a measure of the relevance score from 0.0 to 1.0, where 1.0 means a perfect match. We encourage users to experiment with different values.

In [7]:
#pragma warning disable SKEXP0001, SKEXP0050

var arguments = new KernelArguments();

arguments["fact1"] = "what is my name?";
arguments["fact2"] = "where do I live?";
arguments["fact3"] = "where is my family from?";
arguments["fact4"] = "where have I travelled?";
arguments["fact5"] = "what do I do for work?";

arguments[TextMemoryPlugin.CollectionParam] = MemoryCollectionName;
arguments[TextMemoryPlugin.LimitParam] = "2";
arguments[TextMemoryPlugin.RelevanceParam] = "0.8";

Now that we've included our memories, let's chat!

In [8]:
var history = "";
arguments["history"] = history;
Func<string, Task> Chat = async (string input) => {
    // Save new message in the kernel arguments
    arguments["userInput"] = input;

    // Process the user message and get an answer
    var answer = await chatFunction.InvokeAsync(kernel, arguments);

    // Append the new interaction to the chat history
    var result = $"\nUser: {input}\nChatBot: {answer}\n";

    history += result;
    arguments["history"] = history;
    
    // Show the bot response
    Console.WriteLine(result);
};

In [9]:
await Chat("Hello, I think we've met before, remember? my name is...");


User: Hello, I think we've met before, remember? my name is...
ChatBot: I'm sorry, but I don't have any saved information about you from previous conversations. Could you please tell me your name again?



In [10]:
await Chat("I want to plan a trip and visit my family. Do you know where that is?");


User: I want to plan a trip and visit my family. Do you know where that is?
ChatBot: Yes, you mentioned that your family is from New York. You might want to plan a trip to visit them there! If you need any help with travel arrangements or recommendations for places to visit in New York, feel free to ask.



In [11]:
await Chat("Great! What are some fun things to do there?");


User: Great! What are some fun things to do there?
ChatBot: New York offers a wide variety of activities and attractions to suit every interest. Here are some fun things you can do:

1. **Visit Central Park**: Enjoy a leisurely stroll, go for a bike ride, or have a picnic. There are also plenty of attractions within the park, like the Central Park Zoo and the Conservatory Garden.

2. **Explore Museums**: The Metropolitan Museum of Art, the American Museum of Natural History, and the Museum of Modern Art (MoMA) are world-famous and must-see spots.

3. **See a Broadway Show**: Catching a musical or play on Broadway is a quintessential New York experience.

4. **Statue of Liberty and Ellis Island**: Take a ferry ride to visit these iconic landmarks and learn about the history of immigration to the United States.

5. **Times Square**: Known for its bright lights and bustling atmosphere, Times Square is great for shopping, dining, and people-watching.

6. **Brooklyn Bridge



### Adding documents to your memory

Many times in your applications you'll want to bring in external documents into your memory. Let's see how we can do this using ChromaMemoryStore.

Let's first get some data using some of the links in the Semantic Kernel repo.

In [12]:
const string memoryCollectionName = "SKGitHub";

var githubFiles = new Dictionary<string, string>()
{
    ["https://github.com/microsoft/semantic-kernel/blob/main/README.md"]
        = "README: Installation, getting started, and how to contribute",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/02-running-prompts-from-file.ipynb"]
        = "Jupyter notebook describing how to pass prompts from a file to a semantic plugin or function",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/00-getting-started.ipynb"]
        = "Jupyter notebook describing how to get started with the Semantic Kernel",
    ["https://github.com/microsoft/semantic-kernel/tree/main/prompt_template_samples/ChatPlugin/ChatGPT"]
        = "Sample demonstrating how to create a chat plugin interfacing with ChatGPT",
    ["https://github.com/microsoft/semantic-kernel/blob/main/dotnet/src/Plugins/Plugins.Memory/VolatileMemoryStore.cs"]
        = "C# class that defines a volatile embedding store",
};

Let's build a new Memory.

In [13]:
#pragma warning disable SKEXP0001, SKEXP0010, SKEXP0020, SKEXP0050

var modelId = "text-embedding-ada-002";
ITextEmbeddingGenerationService textEmbeddingService = useAzureOpenAI
    ? new AzureOpenAITextEmbeddingGenerationService(deploymentName: modelId, endpoint: azureEndpoint, credential: new DefaultAzureCredential(defaultAzureCredentialOptions))
    : new OpenAITextEmbeddingGenerationService(modelId: modelId, apiKey: apiKey);

var memory = new SemanticTextMemory(new VolatileMemoryStore(), textEmbeddingService);

Now let's add these files to ChromaMemoryStore using `SaveReferenceAsync`.

In [14]:
Console.WriteLine("Adding some GitHub file URLs and their descriptions to Chroma Semantic Memory.");
var i = 0;
foreach (var entry in githubFiles)
{
    await memory.SaveReferenceAsync(
        collection: memoryCollectionName,
        description: entry.Value,
        text: entry.Value,
        externalId: entry.Key,
        externalSourceName: "GitHub"
    );
    Console.WriteLine($"  URL {++i} saved");
}

Adding some GitHub file URLs and their descriptions to Chroma Semantic Memory.
  URL 1 saved
  URL 2 saved
  URL 3 saved
  URL 4 saved
  URL 5 saved


In [15]:
string ask = "I love Jupyter notebooks, how should I get started?";
Console.WriteLine("===========================\n" +
                    "Query: " + ask + "\n");

var memories = memory.SearchAsync(memoryCollectionName, ask, limit: 5, minRelevanceScore: 0.6);

i = 0;
await foreach (var memory in memories)
{
    Console.WriteLine($"Result {++i}:");
    Console.WriteLine("  URL:     : " + memory.Metadata.Id);
    Console.WriteLine("  Title    : " + memory.Metadata.Description);
    Console.WriteLine("  Relevance: " + memory.Relevance);
    Console.WriteLine();
}

Query: I love Jupyter notebooks, how should I get started?

Result 1:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/00-getting-started.ipynb
  Title    : Jupyter notebook describing how to get started with the Semantic Kernel
  Relevance: 0.8676822185516357

Result 2:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/README.md
  Title    : README: Installation, getting started, and how to contribute
  Relevance: 0.8080047965049744

Result 3:
  URL:     : https://github.com/microsoft/semantic-kernel/blob/main/dotnet/notebooks/02-running-prompts-from-file.ipynb
  Title    : Jupyter notebook describing how to pass prompts from a file to a semantic plugin or function
  Relevance: 0.8019191026687622

Result 4:
  URL:     : https://github.com/microsoft/semantic-kernel/tree/main/prompt_template_samples/ChatPlugin/ChatGPT
  Title    : Sample demonstrating how to create a chat plugin interfacing with ChatGPT
  Relevance: 0.725256860256195